Import necessary libraries

In [2]:
import pandas as pd  # Pandas for data manipulation and analysis
import csv  # CSV module for reading and writing CSV files
from collections import ChainMap  # ChainMap for creating a single view of multiple mappings
import json  # JSON module for working with JSON data

In [3]:
# Open and read a JSON Lines file into a list of dictionaries
with open('nq-train-07.jsonl') as json_file:      
    data = json_file.readlines() # Read lines from the file and store them in a list
    data = list(map(json.loads, data)) # Parse each line as JSON and create a list of dictionaries
df = pd.DataFrame(data) # Create a Pandas DataFrame from the list of dictionaries

Initialize empty dictionaries to store start and end tokens, and a list for indices

In [4]:
start = {}  # Dictionary to store start tokens
end = {}    # Dictionary to store end tokens
index = []  # List to store indices
# Iterate through each row in the DataFrame
for i in range(len(df)):
    # Check if 'short_answers' is present in the annotations for the current row
    if df['annotations'][i][0]['short_answers']:
        index.append(i)  # Append the index to the list
        start[i] = df['annotations'][i][0]['short_answers'][0]['start_token']  # Store the start token
        end[i] = df['annotations'][i][0]['short_answers'][0]['end_token']  # Store the end token

In [5]:
ans = {} # Initialize an empty dictionary to store answers
# Iterate through the indices stored in the 'index' list
for i in index:
    res = '' # Initialize an empty string to concatenate tokens
    # Iterate through the tokens within the specified range [start[i]: end[i]]
    for j in range(len(df['document_tokens'][i][start[i]: end[i]])):
        res += ' ' + df['document_tokens'][i][start[i]: end[i]][j]['token'] # Concatenate each token to the result string
    ans[i] = res # Store the concatenated result as the answer for the corresponding index

Create a new DataFrame from the 'ans' dictionary

In [6]:
df_dictionary = pd.DataFrame(ans.values(), index = ans.keys(), columns = ['answer']) # The values in 'ans' serve as the data, index as row labels, and 'answer' as the column label

Merge the original DataFrame 'df' with the DataFrame 'df_dictionary' using the indices

In [ ]:
fdf = pd.merge(df, df_dictionary, left_index=True, right_index=True) # Left_index and right_index are set to True to perform the merge based on the indices
fdf = fdf[['question_text', 'question_tokens', 'answer']] # Select specific columns ('question_text', 'question_tokens', 'answer') in the merged DataFrame
fdf.to_csv('NQ_Dataset/filename') #Save the resulting DataFrame to a CSV file named 'filename.csv' in the 'NQ_Dataset' directory